In [83]:
from datetime import date
import pandas as pd
import numpy as np
import yaml
from sqlalchemy import create_engine

In [84]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

In [85]:
# Consulta SQL para obtener los servicios y sus estados
query = """
SELECT 
    s.id as servicio_id,
    s.cliente_id,
    s.mensajero_id as mensajero_inicial_id,
    es.estado_id,
    es.fecha as fecha_estado,
    es.hora as hora_estado
FROM mensajeria_servicio s
JOIN mensajeria_estadosservicio es ON s.id = es.servicio_id
ORDER BY s.id, es.fecha, es.hora
"""

In [86]:
# Leer datos y dimensiones
df = pd.read_sql(query, fuente_conn)
dim_fecha = pd.read_sql_table('dim_fecha', bodega_conn)
dim_cliente = pd.read_sql_table('dim_cliente', bodega_conn)
dim_mensajero = pd.read_sql_table('dim_mensajero', bodega_conn)
dim_estado = pd.read_sql_table('dim_estado', bodega_conn)
dim_hora = pd.read_sql_table('dim_hora', bodega_conn)

In [87]:
# Verificaciones iniciales
print("\nTotal registros en tabla de servicios:", len(df))
print("Servicios únicos:", df['servicio_id'].nunique())
print("\nDistribución de estados:")
print(df['estado_id'].value_counts().sort_index())


Total registros en tabla de servicios: 128402
Servicios únicos: 28430

Distribución de estados:
estado_id
1    29627
2    30249
3     5202
4    27542
5    27335
6     8447
Name: count, dtype: int64


In [88]:
# Análisis de estados múltiples
estados_multiples = df.groupby(['servicio_id', 'estado_id']).size().reset_index(name='count')
estados_multiples = estados_multiples[estados_multiples['count'] > 1]
print("\nServicios con estados múltiples:")
print(estados_multiples.groupby('estado_id')['servicio_id'].count())


Servicios con estados múltiples:
estado_id
1    1064
2    2228
3    1086
4     376
5     286
6     106
Name: servicio_id, dtype: int64


In [89]:
# Limpiar formato de hora
def limpiar_hora(hora_str):
    try:
        if '.' in str(hora_str):
            return str(hora_str).split('.')[0]
        return str(hora_str)
    except:
        return None

df['hora_estado'] = df['hora_estado'].apply(limpiar_hora)

In [90]:
print(df['hora_estado'].head())

0    16:22:18
1    17:51:20
2    12:02:48
3    12:16:00
4    17:07:55
Name: hora_estado, dtype: object


In [91]:
print("Tipo de dato fecha_estado:", df['fecha_estado'].dtype)
print("Tipo de dato hora_estado:", df['hora_estado'].dtype)
print("\nEjemplos de fecha_estado:")
print(df['fecha_estado'].head())
print("\nEjemplos de hora_estado:")
print(df['hora_estado'].head())

Tipo de dato fecha_estado: object
Tipo de dato hora_estado: object

Ejemplos de fecha_estado:
0    2023-09-19
1    2023-10-13
2    2023-10-31
3    2023-10-31
4    2023-10-31
Name: fecha_estado, dtype: object

Ejemplos de hora_estado:
0    16:22:18
1    17:51:20
2    12:02:48
3    12:16:00
4    17:07:55
Name: hora_estado, dtype: object


In [92]:
# Antes del pivoteo
print("\nRegistros únicos por servicio_id:", df['servicio_id'].nunique())
print("\nEjemplo de servicios con múltiples estados:")
servicio_ejemplo = df.groupby('servicio_id').size().sort_values(ascending=False).head(1).index[0]
print(df[df['servicio_id'] == servicio_ejemplo].sort_values(['fecha_estado', 'hora_estado']))


Registros únicos por servicio_id: 28430

Ejemplo de servicios con múltiples estados:
        servicio_id  cliente_id  mensajero_inicial_id  estado_id fecha_estado  \
110965        24681          11                  48.0          1   2024-08-05   
110966        24681          11                  48.0          2   2024-08-05   
110967        24681          11                  48.0          4   2024-08-05   
110968        24681          11                  48.0          3   2024-08-05   
110969        24681          11                  48.0          3   2024-08-05   
110970        24681          11                  48.0          3   2024-08-05   
110971        24681          11                  48.0          3   2024-08-05   
110972        24681          11                  48.0          3   2024-08-05   
110973        24681          11                  48.0          3   2024-08-05   
110974        24681          11                  48.0          3   2024-08-05   
110975        24681    

In [93]:
# Para cada estado, tomar el registro más relevante
df_estados = pd.DataFrame()

In [94]:
# Primero procesamos el estado 3 (novedades) para tener la información de novedades disponible
estado_3 = df[df['estado_id'] == 3].groupby('servicio_id').agg({
    'fecha_estado': list,
    'hora_estado': list,
    'cliente_id': 'first',
    'mensajero_inicial_id': 'first'
}).reset_index()

In [95]:
# Crear columna de novedades para todos los servicios
todos_servicios = df['servicio_id'].unique()
df_novedades = pd.DataFrame({'servicio_id': todos_servicios})
df_novedades['tiene_novedad'] = df_novedades['servicio_id'].isin(estado_3['servicio_id'])
df_novedades['cantidad_novedades'] = 0  # Inicializar en 0

In [96]:
# Actualizar cantidad de novedades para servicios que sí tienen
servicios_con_novedades = estado_3.copy()
servicios_con_novedades['cantidad_novedades'] = servicios_con_novedades['fecha_estado'].str.len()
df_novedades.update(
    servicios_con_novedades[['servicio_id', 'cantidad_novedades']].set_index('servicio_id')
)

print("\nDistribución inicial de novedades:")
print(df_novedades['cantidad_novedades'].value_counts().sort_index())


Distribución inicial de novedades:
cantidad_novedades
0     25163
1      2182
2       715
3       220
4        64
5        30
6        17
7         9
8        11
9         6
10        4
11        2
12        1
13        1
14        1
17        1
24        1
30        1
38        1
Name: count, dtype: int64


In [97]:
# Ahora procesamos todos los estados
for estado in [1, 2, 3, 4, 5, 6]:
    if estado == 3:
        estado_data = estado_3.copy()
        estado_data['fecha_primera_novedad'] = estado_data['fecha_estado'].apply(lambda x: x[0] if x else None)
        estado_data['hora_primera_novedad'] = estado_data['hora_estado'].apply(lambda x: x[0] if x else None)
        estado_data['fecha_ultima_novedad'] = estado_data['fecha_estado'].apply(lambda x: x[-1] if x else None)
        estado_data['hora_ultima_novedad'] = estado_data['hora_estado'].apply(lambda x: x[-1] if x else None)
    else:
        estado_data = df[df['estado_id'] == estado].groupby('servicio_id').agg({
            'fecha_estado': 'first',
            'hora_estado': 'first',
            'cliente_id': 'first',
            'mensajero_inicial_id': 'first'
        }).reset_index()
    
    # Renombrar columnas según el estado
    nombre_estado = {
        1: 'iniciado',
        2: 'asignado',
        3: 'novedad',
        4: 'recogido',
        5: 'entregado',
        6: 'cerrado'
    }[estado]
    
    if estado == 3:
        columnas_rename = {
            'fecha_primera_novedad': f'fecha_{nombre_estado}',
            'hora_primera_novedad': f'hora_{nombre_estado}',
            'fecha_ultima_novedad': f'fecha_ultima_{nombre_estado}',
            'hora_ultima_novedad': f'hora_ultima_{nombre_estado}'
        }
    else:
        columnas_rename = {
            'fecha_estado': f'fecha_{nombre_estado}',
            'hora_estado': f'hora_{nombre_estado}'
        }
    
    estado_data = estado_data.rename(columns=columnas_rename)
    
    if len(df_estados) == 0:
        df_estados = estado_data
        # Agregar información de novedades desde el principio
        df_estados = df_estados.merge(df_novedades[['servicio_id', 'cantidad_novedades']], 
                                    on='servicio_id', 
                                    how='left')
    else:
        df_estados = df_estados.merge(estado_data, 
                                    on=['servicio_id', 'cliente_id', 'mensajero_inicial_id'], 
                                    how='outer')

print("\nTotal servicios después de procesar estados:", len(df_estados))
print("\nDistribución final de novedades:")
print(df_estados['cantidad_novedades'].fillna(0).value_counts().sort_index())



Total servicios después de procesar estados: 28430

Distribución final de novedades:
cantidad_novedades
0.0     25164
1.0      2182
2.0       715
3.0       220
4.0        63
5.0        30
6.0        17
7.0         9
8.0        11
9.0         6
10.0        4
11.0        2
12.0        1
13.0        1
14.0        1
17.0        1
24.0        1
30.0        1
38.0        1
Name: count, dtype: int64


In [99]:
# Verificaciones Adicionales
print("\nVerificación de completitud por estado:")
for estado in ['iniciado', 'asignado', 'novedad', 'recogido', 'entregado', 'cerrado']:
    total = len(df_estados[pd.notna(df_estados[f'fecha_{estado}'])])
    print(f"Estado {estado}: {total} registros ({(total/len(df_estados)*100):.2f}%)")


Verificación de completitud por estado:
Estado iniciado: 28429 registros (100.00%)
Estado asignado: 27702 registros (97.44%)
Estado novedad: 3273 registros (11.51%)
Estado recogido: 27016 registros (95.03%)
Estado entregado: 26952 registros (94.80%)
Estado cerrado: 8290 registros (29.16%)


In [100]:
# Función para calcular tiempos
def calcular_tiempo_entre_estados(fecha1, hora1, fecha2, hora2):
    try:
        if pd.isna(fecha1) or pd.isna(fecha2) or pd.isna(hora1) or pd.isna(hora2):
            return "00:00:00"
        
        timestamp1 = pd.to_datetime(f"{fecha1} {hora1}")
        timestamp2 = pd.to_datetime(f"{fecha2} {hora2}")
        
        if timestamp2 < timestamp1:  # Si las fechas están invertidas
            timestamp1, timestamp2 = timestamp2, timestamp1
            
        segundos = (timestamp2 - timestamp1).total_seconds()
        hours = int(segundos // 3600)
        minutes = int((segundos % 3600) // 60)
        seconds = int(segundos % 60)
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    except:
        return "00:00:00"

In [101]:
# Calcular tiempos entre estados
df_estados['tiempo_asignacion'] = df_estados.apply(
    lambda row: calcular_tiempo_entre_estados(
        row['fecha_iniciado'], 
        row['hora_iniciado'],
        row['fecha_asignado'],
        row['hora_asignado']
    ), axis=1
)

# Para novedades, calcular tiempo total en novedades
df_estados['tiempo_total_novedades'] = df_estados.apply(
    lambda row: calcular_tiempo_entre_estados(
        row['fecha_novedad'],
        row['hora_novedad'],
        row['fecha_ultima_novedad'],
        row['hora_ultima_novedad']
    ) if 'tiene_novedad' in row and row['tiene_novedad'] else "00:00:00",
    axis=1
)

df_estados['tiempo_recogida'] = df_estados.apply(
    lambda row: calcular_tiempo_entre_estados(
        row['fecha_asignado'], 
        row['hora_asignado'],
        row['fecha_recogido'],
        row['hora_recogido']
    ) if not row.get('tiene_novedad', False) else
    calcular_tiempo_entre_estados(
        row['fecha_ultima_novedad'],
        row['hora_ultima_novedad'],
        row['fecha_recogido'],
        row['hora_recogido']
    ), axis=1
)

df_estados['tiempo_entrega'] = df_estados.apply(
    lambda row: calcular_tiempo_entre_estados(
        row['fecha_recogido'], 
        row['hora_recogido'],
        row['fecha_entregado'],
        row['hora_entregado']
    ), axis=1
)

df_estados['tiempo_cierre'] = df_estados.apply(
    lambda row: calcular_tiempo_entre_estados(
        row['fecha_entregado'], 
        row['hora_entregado'],
        row['fecha_cerrado'],
        row['hora_cerrado']
    ), axis=1
)

In [102]:
# Verificar cálculo de tiempos
print("\nEjemplo de tiempos calculados para un servicio:")
servicio_ejemplo = df_estados.iloc[0]
print(f"Servicio ID: {servicio_ejemplo['servicio_id']}")
for tiempo in ['tiempo_asignacion', 'tiempo_total_novedades', 'tiempo_recogida', 'tiempo_entrega', 'tiempo_cierre']:
    print(f"{tiempo}: {servicio_ejemplo[tiempo]}")


Ejemplo de tiempos calculados para un servicio:
Servicio ID: 7
tiempo_asignacion: 577:29:02
tiempo_total_novedades: 00:00:00
tiempo_recogida: 426:11:28
tiempo_entrega: 05:05:07
tiempo_cierre: 04:51:55


In [103]:
# Convertir fechas al mismo formato
df_estados['fecha_iniciado'] = pd.to_datetime(df_estados['fecha_iniciado']).dt.date
dim_fecha['fecha'] = pd.to_datetime(dim_fecha['fecha']).dt.date

# Obtener hora del día para dim_hora
df_estados['hora_del_dia'] = df_estados['hora_iniciado'].apply(
    lambda x: int(x.split(':')[0]) if pd.notna(x) else None
)

In [106]:
# Realizar los merges con las dimensiones
hecho_acumulado = df_estados.merge(
    dim_fecha[['key_dim_fecha', 'fecha']], 
    left_on='fecha_iniciado', 
    right_on='fecha',
    how='left'
)
print(f"Registros después de merge con dim_fecha: {len(hecho_acumulado)}")

Registros después de merge con dim_fecha: 28430


In [107]:
hecho_acumulado = hecho_acumulado.merge(
    dim_cliente[['key_dim_cliente', 'cliente_id']], 
    on='cliente_id',
    how='left'
)
print(f"Registros después de merge con dim_cliente: {len(hecho_acumulado)}")

Registros después de merge con dim_cliente: 28430


In [108]:
hecho_acumulado = hecho_acumulado.merge(
    dim_mensajero[['key_dim_mensajero', 'mensajero_id']], 
    left_on='mensajero_inicial_id',
    right_on='mensajero_id',
    how='left'
)
print(f"Registros después de merge con dim_mensajero: {len(hecho_acumulado)}")

Registros después de merge con dim_mensajero: 28430


In [109]:
hecho_acumulado = hecho_acumulado.merge(
    dim_hora[['key_dim_hora', 'hora']], 
    left_on='hora_del_dia',
    right_on='hora',
    how='left'
)
print(f"Registros después de merge con dim_hora: {len(hecho_acumulado)}")

Registros después de merge con dim_hora: 28430


In [110]:
# Reemplazar NaN en cantidad_novedades por 0
hecho_acumulado['cantidad_novedades'] = hecho_acumulado['cantidad_novedades'].fillna(0)

In [111]:
# Seleccionar columnas finales
columnas_finales = [
    'servicio_id',
    'key_dim_fecha',
    'key_dim_cliente',
    'key_dim_mensajero',
    'key_dim_hora',
    'fecha_iniciado',
    'hora_iniciado',
    'fecha_asignado',
    'hora_asignado',
    'fecha_novedad',
    'hora_novedad',
    'fecha_ultima_novedad',
    'hora_ultima_novedad',
    'fecha_recogido',
    'hora_recogido',
    'fecha_entregado',
    'hora_entregado',
    'fecha_cerrado',
    'hora_cerrado',
    'tiempo_asignacion',
    'tiempo_total_novedades',
    'tiempo_recogida',
    'tiempo_entrega',
    'tiempo_cierre',
    'cantidad_novedades'
]

In [112]:
hecho_acumulado = hecho_acumulado[columnas_finales]
# Agregar fecha de carga
hecho_acumulado['saved'] = date.today()

# Verificaciones

In [113]:
# Verificaciones
print("\nInformación del DataFrame:")
print(hecho_acumulado.info())


Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28430 entries, 0 to 28429
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   servicio_id             28430 non-null  int64  
 1   key_dim_fecha           28429 non-null  float64
 2   key_dim_cliente         28430 non-null  int64  
 3   key_dim_mensajero       27703 non-null  float64
 4   key_dim_hora            28429 non-null  float64
 5   fecha_iniciado          28429 non-null  object 
 6   hora_iniciado           28429 non-null  object 
 7   fecha_asignado          27702 non-null  object 
 8   hora_asignado           27702 non-null  object 
 9   fecha_novedad           3273 non-null   object 
 10  hora_novedad            3273 non-null   object 
 11  fecha_ultima_novedad    3273 non-null   object 
 12  hora_ultima_novedad     3273 non-null   object 
 13  fecha_recogido          27016 non-null  object 
 14  hora_recog

In [114]:
print(f"Total registros finales: {len(hecho_acumulado)}")
print("\nDistribución de novedades:")
print(hecho_acumulado['cantidad_novedades'].value_counts().sort_index())

Total registros finales: 28430

Distribución de novedades:
cantidad_novedades
0.0     25164
1.0      2182
2.0       715
3.0       220
4.0        63
5.0        30
6.0        17
7.0         9
8.0        11
9.0         6
10.0        4
11.0        2
12.0        1
13.0        1
14.0        1
17.0        1
24.0        1
30.0        1
38.0        1
Name: count, dtype: int64


In [115]:
hecho_acumulado.head()

,servicio_id,key_dim_fecha,key_dim_cliente,key_dim_mensajero,key_dim_hora,fecha_iniciado,hora_iniciado,fecha_asignado,hora_asignado,fecha_novedad,...,hora_entregado,fecha_cerrado,hora_cerrado,tiempo_asignacion,tiempo_total_novedades,tiempo_recogida,tiempo_entrega,tiempo_cierre,cantidad_novedades,saved
0,7,261.0,7,0.0,16.0,2023-09-19,16:22:18,2023-10-13,17:51:20,NaN,...,17:07:55,2023-10-31,12:16:00,577:29:02,00:00:00,426:11:28,05:05:07,04:51:55,0.0,2024-11-10
1,8,261.0,7,13.0,16.0,2023-09-19,16:30:05,2023-12-20,20:14:43,NaN,...,16:08:35,NaN,NaN,2211:44:38,00:00:00,1339:19:35,1320:34:17,00:00:00,0.0,2024-11-10
2,9,261.0,7,13.0,16.0,2023-09-19,16:30:05,2023-12-28,19:33:01,NaN,...,NaN,NaN,NaN,2403:02:56,00:00:00,00:00:00,00:00:00,00:00:00,0.0,2024-11-10
3,10,261.0,7,13.0,16.0,2023-09-19,16:35:52,2023-12-28,19:33:07,NaN,...,09:58:27,NaN,NaN,2402:57:15,00:00:00,1228:48:40,513:36:40,00:00:00,0.0,2024-11-10
4,11,261.0,7,13.0,16.0,2023-09-19,16:37:54,2023-12-09,13:13:59,NaN,...,NaN,NaN,NaN,1940:36:05,00:00:00,1269:15:56,00:00:00,00:00:00,0.0,2024-11-10


In [116]:
print("\nRegistros con tiempos nulos:")
for tiempo in ['tiempo_asignacion', 'tiempo_recogida', 'tiempo_entrega', 'tiempo_cierre']:
    nulos = hecho_acumulado[tiempo].isna().sum()
    print(f"{tiempo}: {nulos} registros nulos")


Registros con tiempos nulos:
tiempo_asignacion: 0 registros nulos
tiempo_recogida: 0 registros nulos
tiempo_entrega: 0 registros nulos
tiempo_cierre: 0 registros nulos


In [117]:
print("\nVerificar completitud de estados:")
for estado in ['iniciado', 'asignado', 'novedad', 'recogido', 'entregado', 'cerrado']:
    fecha_nulos = hecho_acumulado[f'fecha_{estado}'].isna().sum()
    hora_nulos = hecho_acumulado[f'hora_{estado}'].isna().sum()
    print(f"\nEstado {estado}:")
    print(f"Registros sin fecha: {fecha_nulos}")
    print(f"Registros sin hora: {hora_nulos}")


Verificar completitud de estados:

Estado iniciado:
Registros sin fecha: 1
Registros sin hora: 1

Estado asignado:
Registros sin fecha: 728
Registros sin hora: 728

Estado novedad:
Registros sin fecha: 25157
Registros sin hora: 25157

Estado recogido:
Registros sin fecha: 1414
Registros sin hora: 1414

Estado entregado:
Registros sin fecha: 1478
Registros sin hora: 1478

Estado cerrado:
Registros sin fecha: 20140
Registros sin hora: 20140


In [118]:
print("\nVerificar registros con fechas fuera de secuencia:")
for idx, row in hecho_acumulado.iterrows():
    fechas = [
        (pd.to_datetime(f"{row['fecha_iniciado']} {row['hora_iniciado']}") if pd.notna(row['fecha_iniciado']) else None, 'iniciado'),
        (pd.to_datetime(f"{row['fecha_asignado']} {row['hora_asignado']}") if pd.notna(row['fecha_asignado']) else None, 'asignado'),
        (pd.to_datetime(f"{row['fecha_recogido']} {row['hora_recogido']}") if pd.notna(row['fecha_recogido']) else None, 'recogido'),
        (pd.to_datetime(f"{row['fecha_entregado']} {row['hora_entregado']}") if pd.notna(row['fecha_entregado']) else None, 'entregado'),
        (pd.to_datetime(f"{row['fecha_cerrado']} {row['hora_cerrado']}") if pd.notna(row['fecha_cerrado']) else None, 'cerrado')
    ]
    fechas = [(f, e) for f, e in fechas if f is not None]
    for i in range(len(fechas)-1):
        if fechas[i][0] > fechas[i+1][0]:
            print(f"\nServicio {row['servicio_id']} tiene {fechas[i][1]} después de {fechas[i+1][1]}")
            print(f"{fechas[i][1]}: {row[f'fecha_{fechas[i][1]}']} {row[f'hora_{fechas[i][1]}']}")
            print(f"{fechas[i+1][1]}: {row[f'fecha_{fechas[i+1][1]}']} {row[f'hora_{fechas[i+1][1]}']}")



Verificar registros con fechas fuera de secuencia:

Servicio 7 tiene entregado después de cerrado
entregado: 2023-10-31 17:07:55
cerrado: 2023-10-31 12:16:00

Servicio 22 tiene iniciado después de asignado
iniciado: 2023-09-23 19:19:28
asignado: 2023-09-22 19:45:37

Servicio 23 tiene iniciado después de asignado
iniciado: 2023-09-23 19:19:58
asignado: 2023-09-22 19:45:37

Servicio 24 tiene iniciado después de asignado
iniciado: 2023-09-23 19:22:13
asignado: 2023-09-22 19:45:37

Servicio 25 tiene iniciado después de asignado
iniciado: 2023-09-23 19:22:24
asignado: 2023-09-22 19:45:30

Servicio 26 tiene iniciado después de asignado
iniciado: 2023-09-23 19:25:56
asignado: 2023-09-22 19:45:20

Servicio 83 tiene iniciado después de asignado
iniciado: 2024-01-04 19:07:55
asignado: 2024-01-03 22:20:44

Servicio 86 tiene iniciado después de asignado
iniciado: 2024-01-04 19:31:08
asignado: 2024-01-03 19:31:51

Servicio 87 tiene iniciado después de asignado
iniciado: 2024-01-04 22:11:13
asignad

In [119]:
print("\nEstadísticas de tiempos:")
for tiempo in ['tiempo_asignacion', 'tiempo_total_novedades', 
               'tiempo_recogida', 'tiempo_entrega', 'tiempo_cierre']:
    print(f"\n{tiempo}:")
    print(hecho_acumulado[tiempo].value_counts().head())


Estadísticas de tiempos:

tiempo_asignacion:
tiempo_asignacion
00:00:00    729
00:00:27     97
00:00:20     87
00:00:53     87
00:00:26     85
Name: count, dtype: int64

tiempo_total_novedades:
tiempo_total_novedades
00:00:00    28430
Name: count, dtype: int64

tiempo_recogida:
tiempo_recogida
00:00:00    1414
00:00:29      72
00:00:33      71
00:00:27      65
00:00:38      63
Name: count, dtype: int64

tiempo_entrega:
tiempo_entrega
00:00:00    1479
00:00:21      31
00:00:30      31
00:00:24      29
00:00:35      29
Name: count, dtype: int64

tiempo_cierre:
tiempo_cierre
00:00:00    20141
00:00:29       31
00:00:25       30
00:00:17       27
00:00:23       27
Name: count, dtype: int64


In [120]:
# Guardar en la bodega
hecho_acumulado.to_sql('hecho_entrega_acumulado', bodega_conn, if_exists='replace', index_label='key_hecho_entrega_acumulado')

430

In [121]:
#Pregunta 5: Mensajeros más eficientes (Los que más servicios prestan)
"""
SELECT 
    m.mensajero_id,
    COUNT(*) AS total_servicios,
    AVG(CASE WHEN h.tiempo_asignacion = '00:00:00' THEN NULL 
        ELSE EXTRACT(EPOCH FROM h.tiempo_asignacion::interval)/60 END) AS promedio_minutos_asignacion,
    AVG(CASE WHEN h.tiempo_recogida = '00:00:00' THEN NULL 
        ELSE EXTRACT(EPOCH FROM h.tiempo_recogida::interval)/60 END) AS promedio_minutos_recogida,
    AVG(CASE WHEN h.tiempo_entrega = '00:00:00' THEN NULL 
        ELSE EXTRACT(EPOCH FROM h.tiempo_entrega::interval)/60 END) AS promedio_minutos_entrega,
    AVG(CASE WHEN h.tiempo_cierre = '00:00:00' THEN NULL 
        ELSE EXTRACT(EPOCH FROM h.tiempo_cierre::interval)/60 END) AS promedio_minutos_cierre,
    SUM(CASE WHEN h.cantidad_novedades > 0 THEN 1 ELSE 0 END) AS servicios_con_novedades,
    COUNT(*) FILTER (WHERE fecha_cerrado IS NOT NULL) AS servicios_completados,
    ROUND((COUNT(*) FILTER (WHERE fecha_cerrado IS NOT NULL)::NUMERIC / COUNT(*) * 100), 2) AS porcentaje_completados
FROM 
    hecho_entrega_acumulado h
    JOIN dim_mensajero m ON h.key_dim_mensajero = m.key_dim_mensajero
GROUP BY 
    m.mensajero_id
ORDER BY 
    total_servicios DESC;
"""

"\nSELECT \n    m.mensajero_id,\n    COUNT(*) AS total_servicios,\n    AVG(CASE WHEN h.tiempo_asignacion = '00:00:00' THEN NULL \n        ELSE EXTRACT(EPOCH FROM h.tiempo_asignacion::interval)/60 END) AS promedio_minutos_asignacion,\n    AVG(CASE WHEN h.tiempo_recogida = '00:00:00' THEN NULL \n        ELSE EXTRACT(EPOCH FROM h.tiempo_recogida::interval)/60 END) AS promedio_minutos_recogida,\n    AVG(CASE WHEN h.tiempo_entrega = '00:00:00' THEN NULL \n        ELSE EXTRACT(EPOCH FROM h.tiempo_entrega::interval)/60 END) AS promedio_minutos_entrega,\n    AVG(CASE WHEN h.tiempo_cierre = '00:00:00' THEN NULL \n        ELSE EXTRACT(EPOCH FROM h.tiempo_cierre::interval)/60 END) AS promedio_minutos_cierre,\n    SUM(CASE WHEN h.cantidad_novedades > 0 THEN 1 ELSE 0 END) AS servicios_con_novedades,\n    COUNT(*) FILTER (WHERE fecha_cerrado IS NOT NULL) AS servicios_completados,\n    ROUND((COUNT(*) FILTER (WHERE fecha_cerrado IS NOT NULL)::NUMERIC / COUNT(*) * 100), 2) AS porcentaje_completados\nF

In [122]:
#Pregunta 7: Cuál es el tiempo promedio de entrega desde que se solicita el servicio hasta que se cierra el caso
'''
WITH tiempos_totales AS (
    SELECT 
        h.servicio_id,
        h.fecha_iniciado,
        h.hora_iniciado,
        h.fecha_cerrado,
        h.hora_cerrado,
        CASE 
            WHEN h.fecha_cerrado IS NOT NULL THEN
                EXTRACT(EPOCH FROM (
                    (h.fecha_cerrado || ' ' || h.hora_cerrado)::timestamp - 
                    (h.fecha_iniciado || ' ' || h.hora_iniciado)::timestamp
                ))/60.0
            ELSE NULL
        END AS minutos_totales
    FROM 
        hecho_entrega_acumulado h
    WHERE 
        h.fecha_cerrado IS NOT NULL
)
SELECT 
    COUNT(*) AS total_servicios_completados,
    ROUND(AVG(minutos_totales), 2) AS promedio_minutos,
    ROUND(AVG(minutos_totales)/60, 2) AS promedio_horas,
    ROUND(AVG(minutos_totales)/1440, 2) AS promedio_dias,
    ROUND(MIN(minutos_totales)/60, 2) AS min_horas,
    ROUND(MAX(minutos_totales)/60, 2) AS max_horas
FROM 
    tiempos_totales;
'''

"\nWITH tiempos_totales AS (\n    SELECT \n        h.servicio_id,\n        h.fecha_iniciado,\n        h.hora_iniciado,\n        h.fecha_cerrado,\n        h.hora_cerrado,\n        CASE \n            WHEN h.fecha_cerrado IS NOT NULL THEN\n                EXTRACT(EPOCH FROM (\n                    (h.fecha_cerrado || ' ' || h.hora_cerrado)::timestamp - \n                    (h.fecha_iniciado || ' ' || h.hora_iniciado)::timestamp\n                ))/60.0\n            ELSE NULL\n        END AS minutos_totales\n    FROM \n        hecho_entrega_acumulado h\n    WHERE \n        h.fecha_cerrado IS NOT NULL\n)\nSELECT \n    COUNT(*) AS total_servicios_completados,\n    ROUND(AVG(minutos_totales), 2) AS promedio_minutos,\n    ROUND(AVG(minutos_totales)/60, 2) AS promedio_horas,\n    ROUND(AVG(minutos_totales)/1440, 2) AS promedio_dias,\n    ROUND(MIN(minutos_totales)/60, 2) AS min_horas,\n    ROUND(MAX(minutos_totales)/60, 2) AS max_horas\nFROM \n    tiempos_totales;\n"

In [123]:
#Pregunta 8: Mostrar los tiempos de espera por cada fase del servicio y en qué fase hay más demoras
'''
WITH tiempos_fase AS (
    SELECT 
        'Asignación' as fase,
        COUNT(*) FILTER (WHERE tiempo_asignacion != '00:00:00') as servicios_con_tiempo,
        AVG(EXTRACT(EPOCH FROM tiempo_asignacion::interval)/60) as promedio_minutos,
        MIN(EXTRACT(EPOCH FROM tiempo_asignacion::interval)/60) as min_minutos,
        MAX(EXTRACT(EPOCH FROM tiempo_asignacion::interval)/60) as max_minutos,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY EXTRACT(EPOCH FROM tiempo_asignacion::interval)/60) as mediana_minutos
    FROM hecho_entrega_acumulado
    WHERE tiempo_asignacion != '00:00:00'
    
    UNION ALL
    
    SELECT 
        'Recogida' as fase,
        COUNT(*) FILTER (WHERE tiempo_recogida != '00:00:00') as servicios_con_tiempo,
        AVG(EXTRACT(EPOCH FROM tiempo_recogida::interval)/60) as promedio_minutos,
        MIN(EXTRACT(EPOCH FROM tiempo_recogida::interval)/60) as min_minutos,
        MAX(EXTRACT(EPOCH FROM tiempo_recogida::interval)/60) as max_minutos,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY EXTRACT(EPOCH FROM tiempo_recogida::interval)/60) as mediana_minutos
    FROM hecho_entrega_acumulado
    WHERE tiempo_recogida != '00:00:00'
    
    UNION ALL
    
    SELECT 
        'Entrega' as fase,
        COUNT(*) FILTER (WHERE tiempo_entrega != '00:00:00') as servicios_con_tiempo,
        AVG(EXTRACT(EPOCH FROM tiempo_entrega::interval)/60) as promedio_minutos,
        MIN(EXTRACT(EPOCH FROM tiempo_entrega::interval)/60) as min_minutos,
        MAX(EXTRACT(EPOCH FROM tiempo_entrega::interval)/60) as max_minutos,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY EXTRACT(EPOCH FROM tiempo_entrega::interval)/60) as mediana_minutos
    FROM hecho_entrega_acumulado
    WHERE tiempo_entrega != '00:00:00'
    
    UNION ALL
    
    SELECT 
        'Cierre' as fase,
        COUNT(*) FILTER (WHERE tiempo_cierre != '00:00:00') as servicios_con_tiempo,
        AVG(EXTRACT(EPOCH FROM tiempo_cierre::interval)/60) as promedio_minutos,
        MIN(EXTRACT(EPOCH FROM tiempo_cierre::interval)/60) as min_minutos,
        MAX(EXTRACT(EPOCH FROM tiempo_cierre::interval)/60) as max_minutos,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY EXTRACT(EPOCH FROM tiempo_cierre::interval)/60) as mediana_minutos
    FROM hecho_entrega_acumulado
    WHERE tiempo_cierre != '00:00:00'
)
SELECT 
    fase,
    servicios_con_tiempo,
    CAST(promedio_minutos AS NUMERIC(10,2)) as promedio_minutos,
    CAST(promedio_minutos/60 AS NUMERIC(10,2)) as promedio_horas,
    CAST(min_minutos AS NUMERIC(10,2)) as min_minutos,
    CAST(max_minutos AS NUMERIC(10,2)) as max_minutos,
    CAST(mediana_minutos AS NUMERIC(10,2)) as mediana_minutos
FROM 
    tiempos_fase
ORDER BY 
    promedio_minutos DESC;
'''

# Consulta SQL para ver comportamiento de las novedades sobre los otros estados
'''
WITH categorias_tiempo AS (
    SELECT 
        servicio_id,
        cantidad_novedades,
        EXTRACT(EPOCH FROM tiempo_asignacion::interval)/3600 +
        EXTRACT(EPOCH FROM tiempo_recogida::interval)/3600 +
        EXTRACT(EPOCH FROM tiempo_entrega::interval)/3600 +
        EXTRACT(EPOCH FROM tiempo_cierre::interval)/3600 as horas_totales,
        CASE 
            WHEN cantidad_novedades > 0 THEN 'Con Novedades'
            ELSE 'Sin Novedades'
        END as tipo_servicio
    FROM hecho_entrega_acumulado
    WHERE tiempo_asignacion != '00:00:00'
),
stats AS (
    SELECT 
        tipo_servicio,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY horas_totales) + 
        (PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY horas_totales) - 
         PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY horas_totales)) * 1.5 as limite_superior
    FROM categorias_tiempo
    GROUP BY tipo_servicio
)
SELECT 
    ct.tipo_servicio,
    COUNT(*) as total_servicios,
    CAST(AVG(ct.horas_totales) AS NUMERIC(10,2)) as promedio_horas,
    CAST(MIN(ct.horas_totales) AS NUMERIC(10,2)) as min_horas,
    CAST(MAX(ct.horas_totales) AS NUMERIC(10,2)) as max_horas,
    CAST(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY ct.horas_totales) AS NUMERIC(10,2)) as mediana
FROM categorias_tiempo ct
JOIN stats s ON ct.tipo_servicio = s.tipo_servicio
WHERE ct.horas_totales <= s.limite_superior  -- Eliminar outliers
GROUP BY ct.tipo_servicio
ORDER BY ct.tipo_servicio;
'''

"\nWITH categorias_tiempo AS (\n    SELECT \n        servicio_id,\n        cantidad_novedades,\n        EXTRACT(EPOCH FROM tiempo_asignacion::interval)/3600 +\n        EXTRACT(EPOCH FROM tiempo_recogida::interval)/3600 +\n        EXTRACT(EPOCH FROM tiempo_entrega::interval)/3600 +\n        EXTRACT(EPOCH FROM tiempo_cierre::interval)/3600 as horas_totales,\n        CASE \n            WHEN cantidad_novedades > 0 THEN 'Con Novedades'\n            ELSE 'Sin Novedades'\n        END as tipo_servicio\n    FROM hecho_entrega_acumulado\n    WHERE tiempo_asignacion != '00:00:00'\n),\nstats AS (\n    SELECT \n        tipo_servicio,\n        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY horas_totales) + \n        (PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY horas_totales) - \n         PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY horas_totales)) * 1.5 as limite_superior\n    FROM categorias_tiempo\n    GROUP BY tipo_servicio\n)\nSELECT \n    ct.tipo_servicio,\n    COUNT(*) as total_servicios,\n    